In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AbtBuyBenchmark

benchmark = AbtBuyBenchmark(data_dir_path="../data/")
benchmark

17:32:07 INFO:Extracting Abt-Buy...
17:32:07 INFO:Reading Abt-Buy row_dict...
17:32:07 INFO:Reading Abt-Buy train.csv...
17:32:07 INFO:Reading Abt-Buy valid.csv...
17:32:07 INFO:Reading Abt-Buy test.csv...


<AbtBuyBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Textual/Abt-Buy/abt_buy_exp_data.zip

## Preprocess

In [5]:
attr_list = ['name', 'description', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 100)
    return ' '.join(s_tokens)

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_name': {
        'source_attr': 'name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'description': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_description': {
        'source_attr': 'description',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

17:32:07 INFO:For attr=name, computing actual max_str_len
17:32:07 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:32:07 INFO:For attr=name, using actual_max_str_len=16
17:32:07 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:32:11 INFO:For attr=description, computing actual max_str_len
17:32:11 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:32:11 INFO:For attr=description, using actual_max_str_len=16
17:32:11 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:32:14 INFO:For attr=price, computing actual max_str_len
17:32:14 INFO:actual_max_str_len=9 must be pair to enable NN pooling. Updating to 10
17:32:14 INFO:For attr=price, using actual_max_str_len=10


{'name': NumericalizeInfo(source_attr='name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_name': NumericalizeInfo(source_attr='name', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',

In [11]:
datamodule = benchmark.build_pairwise_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=50,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

17:32:14 INFO:GPU available: True, used: True
17:32:14 INFO:TPU available: None, using: 0 TPU cores
17:32:14 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

17:32:16 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.9 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
5.1 M     Trainable params
2.7 M     Non-trainable params
7.9 M     Total params


17:32:17 WARNING:true_pair_set has 6 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'name': 0.31512534618377686,
 'semantic_name': 0.22283898293972015,
 'description': 0.19615307450294495,
 'semantic_description': 0.1544579267501831,
 'price': 0.11142466962337494}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.3417085427135678,
 'valid_f1_at_0.5': 0.6387959866220736,
 'valid_f1_at_0.7': 0.7777777777777778,
 'valid_f1_at_0.9': 0.39062500000000006,
 'valid_pair_entity_ratio_at_0.3': 2.3980582524271843,
 'valid_pair_entity_ratio_at_0.5': 0.9514563106796117,
 'valid_pair_entity_ratio_at_0.7': 0.5048543689320388,
 'valid_pair_entity_ratio_at_0.9': 0.12135922330097088,
 'valid_precision_at_0.3': 0.20647773279352227,
 'valid_precision_at_0.5': 0.4872448979591837,
 'valid_precision_at_0.7': 0.7740384615384616,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 0.9902912621359223,
 'valid_recall_at_0.5': 0.9271844660194175,
 'valid_recall_at_0.7': 0.7815533980582524,
 'valid_recall_at_0.9': 0.24271844660194175}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.38446969696969696,
  'test_f1_at_0.5': 0.7471264367816092,
  'test_f1_at_0.7': 0.8099999999999999,
  'test_f1_at_0.9': 0.4334600760456273,
  'test_pair_entity_ratio_at_0.3': 2.068126520681265,
  'test_pair_entity_ratio_at_0.5': 0.7688564476885644,
  'test_pair_entity_ratio_at_0.7': 0.4720194647201946,
  'test_pair_entity_ratio_at_0.9': 0.1386861313868613,
  'test_precision_at_0.3': 0.2388235294117647,
  'test_precision_at_0.5': 0.6170886075949367,
  'test_precision_at_0.7': 0.8350515463917526,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 0.9854368932038835,
  'test_recall_at_0.5': 0.9466019417475728,
  'test_recall_at_0.7': 0.7864077669902912,
  'test_recall_at_0.9': 0.2766990291262136}]